In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
from ultralytics import YOLO
import easyocr
import mysql.connector
from datetime import datetime

def connect_to_db():
    try:
        conn = mysql.connector.connect(
            host="localhost",      
            user="root",          
            password="",
            database="license_plate_db"  
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

def save_to_db(plate_text):
    conn = connect_to_db()
    if conn:
        try:
            cursor = conn.cursor()
            
            cursor.execute("SELECT COUNT(*) FROM detections WHERE plate_number = %s", (plate_text,))
            count = cursor.fetchone()[0]
            
            if count == 0:
                sql = "INSERT INTO detections (plate_number) VALUES (%s)"
                val = (plate_text,)
                cursor.execute(sql, val)
                conn.commit()
                print("Hasil disimpan ke database:", plate_text)
            else:
                print("Plat nomor sudah ada, tidak disimpan.")
        
        except mysql.connector.Error as err:
            print(f"Error saat menyimpan ke database: {err}")
        finally:
            cursor.close()
            conn.close()


model = YOLO("C:/Users/gabri/Documents/SourceCode/Model AI/Deteksi-Plat-Nomor-Kendaraan-1/runs/detect/train2/weights/best.pt")

reader = easyocr.Reader(['en']) 

def save_to_file(text, filename="results.txt"):
    with open(filename, "a") as file:
        file.write(text + "\n")

def detect_license_plate():
    cap = cv2.VideoCapture(0)  
    
    if not cap.isOpened():
        print("Error: Kamera tidak bisa dibuka.")
        return

    print("Tekan 'q' untuk keluar...")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Tidak bisa membaca frame dari kamera.")
            break
        
        results = model(frame)

        for result in results:
            for box in result.boxes.xyxy:  
                x1, y1, x2, y2 = map(int, box)
                
                roi = frame[y1:y2, x1:x2]
                
                if roi.size > 0:
                    ocr_result = reader.readtext(roi)
                    for detection in ocr_result:
                        text = detection[1]  
                        confidence = detection[2] 

                        if confidence > 0.8:  
                            print(f"Detected Plate: {text} with Confidence: {confidence}")

                            save_to_file(text)
                            save_to_db(text)
                        
                            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                                        0.9, (0, 255, 0), 2)
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        cv2.imshow("License Plate Detection", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


detect_license_plate()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Tekan 'q' untuk keluar...

0: 480x640 (no detections), 397.4ms
Speed: 7.1ms preprocess, 397.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 370.2ms
Speed: 29.8ms preprocess, 370.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 358.2ms
Speed: 4.1ms preprocess, 358.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 423.8ms
Speed: 0.5ms preprocess, 423.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 312.1ms
Speed: 4.3ms preprocess, 312.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 319.0ms
Speed: 1.0ms preprocess, 319.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 210.1ms
Speed: 0.0ms preprocess, 210.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections),